# Fifth Day of Python 4 DIP

> Contents:
> 1. Images as Matrix + Manipulation on Images
> 2. Point evaluations of images


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
Image = plt.imread('./Figures/cameraman.jpg')[...,0]
plt.imshow(Image)
plt.colorbar()
plt.axis('off')
plt.show()

### Basic manipulations
### 1. Cropping

In [ ]:
Image_Cropped = Image[30:100,80:150]
plt.imshow(Image_Cropped)
plt.axis('off')
plt.colorbar()
plt.show()

### 2. Dimming 

In [ ]:
Image_Dimmed = (Image/Image.max())**2*255
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(12,7))
A = ax[0].imshow(Image)
ax[0].axis('off')
ax[0].set_title("Original")

B = ax[1].imshow(Image_Dimmed)
ax[1].axis('off')
ax[1].set_title("Dimmed")
fig.colorbar(A)
fig.colorbar(B)
plt.show()

### 3. Resizing

In [ ]:
M,N = Image.shape
Mr,Nr = (64,64)

ScaleX = M/Mr
ScaleY = N/Nr

Image_Resized = np.array([Image[int(i*ScaleX),int(j*ScaleY)] for i in range(Mr) for j in range(Nr)]).reshape(Mr,Nr)
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(12,7))
ax[0].imshow(Image)
ax[0].axis('off')
ax[0].set_title("Original")

ax[1].imshow(Image_Resized)
ax[1].axis('off')
ax[1].set_title("Resized")

### 4. Rotation

$R = \begin{bmatrix} cos(\theta) & -sin(\theta)\\ sin(\theta) & cos(\theta) \end{bmatrix}$

In [ ]:
M,N = Image.shape
Image_Rotated = np.zeros_like(Image)
Mr,Nr = Image_Rotated.shape
theta = -np.pi/4
R = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])

center = np.array([int(Mr/2),int(Nr/2)])
for i in range(M):
    for j in range(N):
        pixel = np.array([i,j])
        P = np.matmul(R,pixel-center) + center
        if (0 <= P[0] <= Mr and 0 <= P[1] <= Nr):
            Image_Rotated[i,j] = Image[int(P[0]),int(P[1])]
        else:
            Image_Rotated[i,j] = 255



fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(12,7))
ax[0].imshow(Image)
ax[0].axis('off')
ax[0].set_title("Original")

ax[1].imshow(Image_Rotated)
ax[1].axis('off')
ax[1].set_title("Rotated")

### The process of histogram calculation

In [ ]:
def Histogram(X,bins=32):
    """Histogram calculator"""
    Uniqed = np.unique(X.flatten())
    batch = int(Uniqed.max()/bins)
    Bins = np.zeros((bins,1))

    for j in range(bins):
        for i in range(len(Uniqed)):
            if  Uniqed[i] < batch * j:
                Bins[j,:] = np.count_nonzero(X == Uniqed[i])
    Cdf = Bins.cumsum()
    return Bins, Cdf/Cdf.max()

In [ ]:
Hist,CDF = Histogram(Image,bins=128)
Bins = list(range(len(Hist)))
fig, ax = plt.subplots(nrows=1,ncols=3,figsize=(18,7))
ax[0].imshow(Image)
ax[0].axis('off')

ax[1].stem(Bins,np.sort(Hist))
ax[1].set_title("Histogram")

ax[2].plot(Bins,CDF)
ax[2].set_title("CDF")


### Alternative way with matplotlib

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(18,7))
ax[0].imshow(Image)
ax[0].axis('off')
A = ax[1].hist(Image.ravel(),Bins,ec='black',fc='green')
B = ax[2].hist(Image.ravel(),Bins,ec='black',fc='green',cumulative=True)

### Using Numpy

In [ ]:
Hist2,Bins2 = np.histogram(Image.ravel(),bins=range(128))
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(18,7))
ax[0].imshow(Image)
ax[0].axis('off')

ax[1].stem(Bins2[:-1],Hist2)
ax[2].plot(Bins2[:-1],Hist2.cumsum()/Hist2.sum())
plt.show()

In [ ]:
Temp1 = np.zeros((64,64))
Temp2 = Temp1.copy()

## creation of images
Temp1[:,:32] = 1.0

for i in range(64):
    for j in range(64):
        if (i % 2 == 0):
            Temp2[i,j] = 1.0

Hist3,Bins3 = np.histogram(Temp1.ravel(),bins=range(64))
Hist4,Bins4 = np.histogram(Temp2.ravel(),bins=range(64))

fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(16,8))
ax[0,0].imshow(Temp1)
ax[0,0].set_title("Image1")

ax[1,0].imshow(Temp2)
ax[1,0].set_title("Image2")

ax[0,1].stem(Bins3[:-1],Hist3)
ax[0,1].set_title("Histogram1")

ax[1,1].stem(Bins4[:-1],Hist4)
ax[1,1].set_title("Histogram2")


### Brightness of image
$$B(I) = \frac{1}{wh}\sum\limits_{v=1}^{h}\sum\limits_{u=1}^{w} I(u,v)$$

In [ ]:
def Brightness(Image):
    w,h = Image.shape
    return 1/(w*h)*np.sum(np.sum(Image,axis=1),axis=0)

In [ ]:
Brightness(Image)

### Michalson's equation for contrast
$$C_M(I) = \frac{max(I) - min(I)}{max(I)+min(I)}$$

In [ ]:
Contrast = lambda I: (I.max() - I.min())/(I.max() + I.min()) 
Contrast(Image)

### Gray Level Transformations
* `Linear`
* `Logarithmic`
* `Power law`

In [ ]:
Logarithmic = lambda c,r: c*np.log(1+r)
Power_Law = lambda c,r,gamma: c*r**gamma
Image_Trans_Log = Logarithmic(4,Image) 
Image_Trans_Pow = Power_Law(4,Image,0.51)
Hist5,Bins5 = np.histogram(Image.ravel(),bins=range(128))
Hist6,Bins6 = np.histogram(Image_Trans_Log.ravel(),bins=range(128))
Hist7,Bins7 = np.histogram(Image_Trans_Pow.ravel(),bins=range(128))

fig,ax = plt.subplots(nrows=3,ncols=2,figsize=(12,10))
ax[0,0].imshow(Image)
ax[0,0].set_title("Original")

ax[1,0].imshow(Image_Trans_Log)
ax[1,0].set_title("Log")

ax[2,0].imshow(Image_Trans_Pow)
ax[2,0].set_title("Pow")


ax[0,1].stem(Bins5[:-1],Hist5)
ax[0,1].set_title("Original")

ax[1,1].stem(Bins6[:-1],Hist6)
ax[1,1].set_title("Log")

ax[2,1].stem(Bins7[:-1],Hist7)
ax[2,1].set_title("Pow")